<h1>Токены и вложения токенов</h1>
<i>Изучение токенов и вложений как неотъемлемой части построения LLM</i>

### [НЕОБЯЗАТЕЛЬНО]
Если вы просматриваете этот блокнот в Google Colab (или любом другом поставщике облачных услуг), вам нужно **раскомментировать и запустить** следующий блок кода, чтобы установить зависимости:

---

💡 **ПРИМЕЧАНИЕ**: для запуска примеров в этом блокноте нам понадобится графический процессор. В Google Colab перейдите в
**Среда выполнения > Изменить тип среды выполнения > Аппаратный ускоритель > Графический процессор > Тип графического процессора > T4**.

---

In [ ]:
%%capture
!pip install transformers>=4.41.2 sentence-transformers>=3.0.1 gensim>=4.3.2 scikit-learn>=1.5.0 accelerate>=0.31.0

# Загрузка и запуск LLM

Первый шаг — загрузить нашу модель на GPU для более быстрого вывода. Обратите внимание, что мы загружаем модель и токенизатор отдельно и сохраняем их как таковые, чтобы мы могли исследовать их по отдельности.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
!pip install transformers==4.40.2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 144.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 126.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.1
    Uninstalling transformers-4.57.1:
      Successfully uninstalled transformers-4.57.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.1.2 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.40.2 which is incompatible.


In [ ]:
prompt = "Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened.<|assistant|>"

# Tokenize the input prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")

# Generate the text
generation_output = model.generate(
  input_ids=input_ids,
  max_new_tokens=20
)

# Print the output
print(tokenizer.decode(generation_output[0]))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


AttributeError: 'DynamicCache' object has no attribute 'seen_tokens'

In [ ]:
prompt = "Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened."

chat = [{"role": "user", "content": prompt}]
input_ids = tokenizer.apply_chat_template(chat, return_tensors="pt").to("cuda")

outputs = model.generate(input_ids, max_new_tokens=100, use_cache=False)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened. 
 I need a CMake script for setting up a build environment for a C++ project. The script should specify the use of the GNU C++ compiler and define a set of preprocessor definitions for various features like logging, shared libraries, and threading. It should also include paths for header files, which are relative to the project's source directory. The script should list dependencies for C++ source files, ensuring that changes in these files trigger recompilation. Additionally, it


In [ ]:
print(input_ids)

tensor([[32010, 14350,   385,  4876, 27746,  5281,   304, 19235,   363,   278,
         25305,   293, 16423,   292,   286,   728,   481, 29889, 12027,  7420,
           920,   372,  9559, 29889, 32007, 32000]], device='cuda:0')


In [ ]:
for id in input_ids[0]:
   print(tokenizer.decode(id))

<|user|>
Write
an
email
apolog
izing
to
Sarah
for
the
trag
ic
garden
ing
m
ish
ap
.
Exp
lain
how
it
happened
.
<|end|>
<|endoftext|>


In [ ]:
outputs

tensor([[32010, 14350,   385,  4876, 27746,  5281,   304, 19235,   363,   278,
         25305,   293, 16423,   292,   286,   728,   481, 29889, 12027,  7420,
           920,   372,  9559, 29889, 32007, 32000, 29871,    13, 32010,   306,
           817,   263,   315,  9984,  2471,   363,  4444,   701,   263,  2048,
          5177,   363,   263,   315,  1817,  2060, 29889,   450,  2471,   881,
          6084,   278,   671,   310,   278, 15143,   315,  1817,  6516,   322,
          4529,   263,   731,   310,   758, 26482, 15848,   363,  5164,  5680,
           763, 12183, 29892,  7258,  9562, 29892,   322,  3244,   292, 29889,
           739,   881,   884,  3160, 10898,   363,  4839,  2066, 29892,   607,
           526,  6198,   304,   278,  2060, 29915, 29879,  2752,  3884, 29889,
           450,  2471,   881,  1051,  9962,   363,   315,  1817,  2752,  2066,
         29892,  5662,  3864,   393,  3620,   297,  1438,  2066,  7135,   337,
          2388,  8634, 29889, 19814, 29892,   372]],

In [ ]:
print(tokenizer.decode(3323))
print(tokenizer.decode(622))
print(tokenizer.decode([3323, 622]))
print(tokenizer.decode(29901))

Sub
ject
Subject
:


# Сравнение обученных токенизаторов LLM

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

colors_list = [
    '102;194;165', '252;141;98', '141;160;203',
    '231;138;195', '166;216;84', '255;217;47'
]

def show_tokens(sentence, tokenizer_name):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    token_ids = tokenizer(sentence).input_ids
    for idx, t in enumerate(token_ids):
        print(
            f'\x1b[0;30;48;2;{colors_list[idx % len(colors_list)]}m' +
            tokenizer.decode(t) +
            '\x1b[0m',
            end=' '
        )

In [ ]:
text = """
English and CAPITALIZATION
🎵 鸟
show_tokens False None elif == >= else: two tabs:"    " Three tabs: "       "
12.0*50=600
"""

In [ ]:
show_tokens(text, "bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

[CLS] english and capital ##ization [UNK] [UNK] show _ token ##s false none eli ##f = = > = else : two tab ##s : " " three tab ##s : " " 12 . 0 * 50 = 600 [SEP] 

In [ ]:
show_tokens(text, "bert-base-cased")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

[CLS] English and CA ##PI ##TA ##L ##I ##Z ##AT ##ION [UNK] [UNK] show _ token ##s F ##als ##e None el ##if = = > = else : two ta ##bs : " " Three ta ##bs : " " 12 . 0 * 50 = 600 [SEP] 

In [ ]:
show_tokens(text, "gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]


 English  and  CAP ITAL IZ ATION 
 � � �  � � � 
 show _ t ok ens  False  None  el if  ==  >=  else :  two  tabs :"        "  Three  tabs :  "              " 
 12 . 0 * 50 = 600 
 

In [ ]:
show_tokens(text, "google/flan-t5-small")

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

English and CA PI TAL IZ ATION  <unk>  <unk> show _ to ken s Fal s e None  e l if = = > = else : two tab s : " " Three tab s : " " 12. 0 * 50 = 600  </s> 

In [ ]:
# The official is `tiktoken` but this the same tokenizer on the HF platform
show_tokens(text, "Xenova/gpt-4")

tokenizer_config.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/98.0 [00:00<?, ?B/s]


 English  and  CAPITAL IZATION 
 � � �  � � � 
 show _tokens  False  None  elif  ==  >=  else :  two  tabs :"      "  Three  tabs :  "         "
 12 . 0 * 50 = 600 
 

In [ ]:
# You need to request access before being able to use this tokenizer
show_tokens(text, "bigcode/starcoder2-15b")

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]


 English  and  CAPITAL IZATION 
 � � �   � � 
 show _ tokens  False  None  elif  ==  >=  else :  two  tabs :"      "  Three  tabs :  "         " 
 1 2 . 0 * 5 0 = 6 0 0 
 

In [ ]:
show_tokens(text, "facebook/galactica-1.3b")

tokenizer_config.json:   0%|          | 0.00/166 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.00 [00:00<?, ?B/s]


 English  and  CAP ITAL IZATION 
 � � � �  � � � 
 show _ tokens  False  None  elif   ==   > =  else :  two  t abs : "      "  Three  t abs :   "         " 
 1 2 . 0 * 5 0 = 6 0 0 
 

In [ ]:
show_tokens(text, "microsoft/Phi-3-mini-4k-instruct")

 
 English and C AP IT AL IZ ATION 
 � � � �  � � � 
 show _ to kens False None elif == >= else : two tabs :"    " Three tabs : "       " 
 1 2 . 0 * 5 0 = 6 0 0 
 

# Контекстуализированные эмбединги слов из языковой модели (например, BERT)

In [ ]:
from transformers import AutoModel, AutoTokenizer

# Load a tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base")

# Load a language model
model = AutoModel.from_pretrained("microsoft/deberta-v3-xsmall")

# Tokenize the sentence
tokens = tokenizer('Hello world', return_tensors='pt')

# Process the tokens
output = model(**tokens)[0]

In [ ]:
output.shape

torch.Size([1, 4, 384])

In [ ]:
for token in tokens['input_ids'][0]:
    print(tokenizer.decode(token))

[CLS]
Hello
 world
[SEP]


In [ ]:
output

tensor([[[-3.4816,  0.0861, -0.1819,  ..., -0.0612, -0.3911,  0.3017],
         [ 0.1898,  0.3208, -0.2315,  ...,  0.3714,  0.2478,  0.8048],
         [ 0.2071,  0.5036, -0.0485,  ...,  1.2175, -0.2292,  0.8582],
         [-3.4278,  0.0645, -0.1427,  ...,  0.0658, -0.4367,  0.3834]]],
       grad_fn=<NativeLayerNormBackward0>)

# Эмбеддинги текста (для предложений и целых документов)

In [ ]:
from sentence_transformers import SentenceTransformer

# Load model
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# Convert text to text embeddings
vector = model.encode("Best movie ever!")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
vector.shape

(768,)

# Word Embeddings до LLMs


In [ ]:
!pip install gensim

In [ ]:
import gensim.downloader as api

# Download embeddings (66MB, glove, trained on wikipedia, vector size: 50)
# Other options include "word2vec-google-news-300"
# More options at https://github.com/RaRe-Technologies/gensim-data
model = api.load("glove-wiki-gigaword-50")

[==================================================] 100.0% 66.0/66.0MB downloaded


In [ ]:
model.most_similar([model['king']], topn=11)

[('king', 1.0000001192092896),
 ('prince', 0.8236179351806641),
 ('queen', 0.7839043140411377),
 ('ii', 0.7746230363845825),
 ('emperor', 0.7736247777938843),
 ('son', 0.766719400882721),
 ('uncle', 0.7627150416374207),
 ('kingdom', 0.7542161345481873),
 ('throne', 0.7539914846420288),
 ('brother', 0.7492411136627197),
 ('ruler', 0.7434253692626953)]

# # Рекомендация песен по embeddings

In [ ]:
import pandas as pd
from urllib import request

# Get the playlist dataset file
data = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/train.txt')

# Parse the playlist dataset file. Skip the first two lines as
# they only contain metadata
lines = data.read().decode("utf-8").split('\n')[2:]

# Remove playlists with only one song
playlists = [s.rstrip().split() for s in lines if len(s.split()) > 1]

# Load song metadata
songs_file = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/song_hash.txt')
songs_file = songs_file.read().decode("utf-8").split('\n')
songs = [s.rstrip().split('\t') for s in songs_file]
songs_df = pd.DataFrame(data=songs, columns = ['id', 'title', 'artist'])
songs_df.index = songs_df.index.astype(str)
songs_df = songs_df.set_index('id')

In [ ]:
print( 'Playlist #1:\n ', playlists[0], '\n')
print( 'Playlist #2:\n ', playlists[1])

Playlist #1:
  ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '2', '42', '43', '44', '45', '46', '47', '48', '20', '49', '8', '50', '51', '52', '53', '54', '55', '56', '57', '25', '58', '59', '60', '61', '62', '3', '63', '64', '65', '66', '46', '47', '67', '2', '48', '68', '69', '70', '57', '50', '71', '72', '53', '73', '25', '74', '59', '20', '46', '75', '76', '77', '59', '20', '43'] 

Playlist #2:
  ['78', '79', '80', '3', '62', '81', '14', '82', '48', '83', '84', '17', '85', '86', '87', '88', '74', '89', '90', '91', '4', '73', '62', '92', '17', '53', '59', '93', '94', '51', '50', '27', '95', '48', '96', '97', '98', '99', '100', '57', '101', '102', '25', '103', '3', '104', '105', '106', '107', '47', '108', '109', '110', '111', '112', '113', '25', '63', '62', '114', '115', '84', '116', '117',

In [ ]:
from gensim.models import Word2Vec

# Train our Word2Vec model
model = Word2Vec(
    playlists, vector_size=32, window=20, negative=50, min_count=1, workers=4
)

In [ ]:
song_id = 2172

# Ask the model for songs similar to song #2172
model.wv.most_similar(positive=str(song_id))

[('2849', 0.9963427186012268),
 ('3116', 0.9962995648384094),
 ('6626', 0.9962118268013),
 ('3119', 0.995599091053009),
 ('5586', 0.9953666925430298),
 ('10072', 0.9952837228775024),
 ('11473', 0.995011568069458),
 ('6658', 0.9938790202140808),
 ('1849', 0.9932984113693237),
 ('11477', 0.9924756288528442)]

In [ ]:
print(songs_df.iloc[2172])

title     Fade To Black
artist        Metallica
Name: 2172 , dtype: object


***Допишите функцию рекомендации похожих песен***

In [ ]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 59.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from urllib import request
from gensim.models import Word2Vec

# === Загрузка плейлистов (как у вас было) ===
data = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/train.txt')
lines = data.read().decode("utf-8").split('\n')[2:]
playlists = [s.rstrip().split() for s in lines if len(s.split()) > 1]

# === Загрузка метаданных песен ===
songs_file = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/song_hash.txt')
songs_lines = songs_file.read().decode("utf-8").split('\n')
songs = [s.rstrip().split('\t') for s in songs_lines if s.strip()]

# Создаем DataFrame: колонка 'id' содержит хеш (или строку из файла), но
# ВАЖНО: делаем reset_index чтобы индекс DataFrame соответствовал позициям (0..N-1)
songs_df = pd.DataFrame(data=songs, columns=['id', 'title', 'artist']).dropna().reset_index(drop=True)

# Для удобства: приведём индекс к строковому типу, т.к. в playlists элементы тоже строки
songs_df.index = songs_df.index.astype(str)

print('Песен в датасете:', len(songs_df))
print('Пример строки (index -> id, title, artist):')
print(songs_df.iloc[0])

# === Обучаем/используем модель Word2Vec ===
# (Если модель уже обучена — этот шаг можно пропустить)
model = Word2Vec(playlists, vector_size=32, window=20, negative=50, min_count=1, workers=4)

# === Функция рекомендаций, возвращающая DataFrame и печатающая результат ===
def get_recommendations_df(song_id, topn=10):
    sid = str(song_id)  # playlist IDs — строки в ваших списках
    if sid not in model.wv:
        raise KeyError(f"Song ID {sid} not in model vocabulary")
    sim = model.wv.most_similar(positive=[sid], topn=topn)
    recs = []
    for sim_id, score in sim:
        # sim_id — тоже строка номер-позиция; пытаемся взять строку из songs_df
        if sim_id in songs_df.index:
            row = songs_df.loc[sim_id]
            recs.append({'song_index': sim_id, 'title': row['title'], 'artist': row['artist'], 'score': score})
        else:
            recs.append({'song_index': sim_id, 'title': None, 'artist': None, 'score': score})
    return pd.DataFrame(recs)

def print_recommendations(song_id, topn=10):
    sid = str(song_id)
    print()
    if sid in songs_df.index:
        base = songs_df.loc[sid]
        print(f"🎵 Recommendations for: [{sid}] {base['title']} — {base['artist']}\n")
    else:
        print(f"🎵 Recommendations for Song ID {sid} (metadata not found)\n")

    df = get_recommendations_df(song_id, topn=topn)
    for _, r in df.iterrows():
        if pd.notna(r['title']):
            print(f"  ▶ [{r['song_index']}] {r['title']} — {r['artist']} (similarity: {r['score']:.3f})")
        else:
            print(f"  ▶ [{r['song_index']}] [metadata not found] (similarity: {r['score']:.3f})")

# === Примеры ===
print_recommendations(2172)
print()
print_recommendations(842)


Песен в датасете: 75261
Пример строки (index -> id, title, artist):
id                                  0 
title     Gucci Time (w\/ Swizz Beatz)
artist                      Gucci Mane
Name: 0, dtype: object

🎵 Recommendations for: [2172] Fade To Black — Metallica

  ▶ [10084] Detroit Rock City — Kiss (similarity: 0.997)
  ▶ [6658] (Bang Your Head) Metal Health — Quiet Riot (similarity: 0.997)
  ▶ [2976] I Don't Know — Ozzy Osbourne (similarity: 0.996)
  ▶ [1922] One — Metallica (similarity: 0.996)
  ▶ [3117] Still Of The Night — Whitesnake (similarity: 0.996)
  ▶ [2849] Run To The Hills — Iron Maiden (similarity: 0.996)
  ▶ [5586] The Last In Line — Dio (similarity: 0.996)
  ▶ [5549] November Rain — Guns N' Roses (similarity: 0.995)
  ▶ [11473] Little Guitars — Van Halen (similarity: 0.994)
  ▶ [3126] Heavy Metal — Sammy Hagar (similarity: 0.994)


🎵 Recommendations for: [842] California Love (w\/ Dr. Dre & Roger Troutman) — 2Pac

  ▶ [9666] Gimme The Light — Sean Paul (similarity: 0.

In [ ]:
print_recommendations(2172)


🎵 Recommendations for: [2172] Fade To Black — Metallica

  ▶ [10084] Detroit Rock City — Kiss (similarity: 0.997)
  ▶ [6658] (Bang Your Head) Metal Health — Quiet Riot (similarity: 0.997)
  ▶ [2976] I Don't Know — Ozzy Osbourne (similarity: 0.996)
  ▶ [1922] One — Metallica (similarity: 0.996)
  ▶ [3117] Still Of The Night — Whitesnake (similarity: 0.996)
  ▶ [2849] Run To The Hills — Iron Maiden (similarity: 0.996)
  ▶ [5586] The Last In Line — Dio (similarity: 0.996)
  ▶ [5549] November Rain — Guns N' Roses (similarity: 0.995)
  ▶ [11473] Little Guitars — Van Halen (similarity: 0.994)
  ▶ [3126] Heavy Metal — Sammy Hagar (similarity: 0.994)


In [ ]:
print_recommendations(842)


🎵 Recommendations for: [842] California Love (w\/ Dr. Dre & Roger Troutman) — 2Pac

  ▶ [9666] Gimme The Light — Sean Paul (similarity: 0.997)
  ▶ [12205] Give It Up To Me — Sean Paul (similarity: 0.997)
  ▶ [196] I'll Be Missing You — Puff Daddy & The Family (similarity: 0.997)
  ▶ [413] If I Ruled The World (Imagine That) (w\/ Lauryn Hill) — Nas (similarity: 0.997)
  ▶ [5668] How We Do (w\/ 50 Cent) — The Game (similarity: 0.997)
  ▶ [886] Heartless — Kanye West (similarity: 0.996)
  ▶ [12335] King Of The Dancehall — Beenie Man (similarity: 0.996)
  ▶ [5681] Drop It Low (w\/ Chris Brown) — Ester Dean (similarity: 0.995)
  ▶ [1560] In Da Club — 50 Cent (similarity: 0.995)
  ▶ [11331] Take You There — Sean Kingston (similarity: 0.995)
